In [2]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Joins and Data Partitions")
    .master("local[*]")
    .getOrCreate()
)

spark

In [3]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

In [4]:
# Create emp & dept DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [6]:
# Show emp dataframe (ACTION)

emp.show()
dept.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [7]:
emp.rdd.getNumPartitions()

16

In [8]:
emp_partioned = emp.repartition(100)

In [9]:
emp_partioned.rdd.getNumPartitions()

100

In [10]:
emp_partitioned = emp.coalesce(100)

In [12]:
emp_partitioned.rdd.getNumPartitions()

16

In [13]:
from pyspark.sql.functions import spark_partition_id

emp_1 = emp.withColumn("partition_num", spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|            0|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|            1|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            2|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            3|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|            3|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|            4|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|            5|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            6|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-0

In [14]:
emp_1 = emp.repartition(4, "department_id").withColumn("partition_num", spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-0

In [15]:
df_joined = emp.join(dept, how="inner", on=emp.department_id==dept.department_id)

In [17]:
df_joined.select(emp.name,dept.department_id, dept.department_name, emp.salary).show()

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|James Johnson|          101|               Sales| 70000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|     Kate Kim|          102|           Marketing| 51000|
|    Grace Kim|          102|           Marketing| 53000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|      Tom Tan|          103|             Finance| 58000|
|  Steven Chen|          103|             Finance| 62000|
|     Lisa Lee|          104|         Engineering| 47000|
|   David Park|          104|         Engineering| 65000|
|    Nancy Liu|          104|         Engineering| 50000|
|   Susan Chen

In [18]:
df_joined = emp.alias("e").join(dept.alias("d"), how="inner", on=emp.department_id==dept.department_id)
df_joined.select("e.name", "d.department_id", "d.department_name", "e.salary").show()

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|James Johnson|          101|               Sales| 70000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|     Kate Kim|          102|           Marketing| 51000|
|    Grace Kim|          102|           Marketing| 53000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|      Tom Tan|          103|             Finance| 58000|
|  Steven Chen|          103|             Finance| 62000|
|     Lisa Lee|          104|         Engineering| 47000|
|   David Park|          104|         Engineering| 65000|
|    Nancy Liu|          104|         Engineering| 50000|
|   Susan Chen

In [19]:
df_joined = emp.alias("e").join(dept.alias("d"), how="left_outer", on=emp.department_id==dept.department_id)
df_joined.select("e.name", "d.department_id", "d.department_name", "e.salary").show()

+-------------+-------------+--------------------+------+
|         name|department_id|     department_name|salary|
+-------------+-------------+--------------------+------+
|     John Doe|          101|               Sales| 50000|
|   Jane Smith|          101|               Sales| 45000|
|    Bob Brown|          102|           Marketing| 55000|
|    Alice Lee|          102|           Marketing| 48000|
|    Jack Chan|          103|             Finance| 60000|
|    Jill Wong|          103|             Finance| 52000|
|James Johnson|          101|               Sales| 70000|
|     Kate Kim|          102|           Marketing| 51000|
|     Lisa Lee|          104|         Engineering| 47000|
|      Tom Tan|          103|             Finance| 58000|
|   David Park|          104|         Engineering| 65000|
|   Susan Chen|          105|     Human Resources| 54000|
|    Brian Kim|          106|Research and Deve...| 75000|
|    Emily Lee|          107|    Customer Service| 46000|
|  Michael Lee

In [25]:
df_joined = emp.join(dept, how="left_outer", 
                                on=(emp.department_id==dept.department_id) 
                                 & ((emp.department_id == "101") | (emp.department_id == "102"))
                                 & (emp.salary.isNotNull())
)
df_joined.select(emp.name,dept.department_id, dept.department_name, emp.salary).show()

+-------------+-------------+---------------+------+
|         name|department_id|department_name|salary|
+-------------+-------------+---------------+------+
|     John Doe|          101|          Sales| 50000|
|   Jane Smith|          101|          Sales| 45000|
|    Bob Brown|          102|      Marketing| 55000|
|    Alice Lee|          102|      Marketing| 48000|
|    Jack Chan|         null|           null| 60000|
|    Jill Wong|         null|           null| 52000|
|James Johnson|          101|          Sales| 70000|
|     Kate Kim|          102|      Marketing| 51000|
|     Lisa Lee|         null|           null| 47000|
|      Tom Tan|         null|           null| 58000|
|   David Park|         null|           null| 65000|
|   Susan Chen|         null|           null| 54000|
|    Brian Kim|         null|           null| 75000|
|    Emily Lee|         null|           null| 46000|
|  Michael Lee|         null|           null| 63000|
|  Kelly Zhang|         null|           null| 